# Function Decorators and Closures
Function decorators let us “mark” functions in the source code to enhance their behavior in some way. This is powerful stuff, but mastering it requires understanding closures.

One of the newest reserved keywords in Python is nonlocal, introduced in Python 3.0. You can have a profitable life as a Python programmer without ever using it if you adhere to a strict regimen of class-centered object orientation. However, if you want to implement your own function decorators, you must know closures inside out, and then the need for nonlocal becomes obvious.

In [2]:
# we can implement something like this
def double(x):
    return 2 * x

def add(a, b):
    return a + b

double(add(1, 2))

6

In [3]:
# as this
def double(func):
    def inner(*args, **kwargs):
        return 2 * func(*args, **kwargs)
    return inner

@double
def add(a, b):
    return a + b

add(1, 2)

6

In [4]:
add

<function __main__.double.<locals>.inner(*args, **kwargs)>

Remember to use functools.wraps for this issue.

In [5]:
import functools

def double(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        return 2 * func(*args, **kwargs)
    return inner

@double
def add(a, b):
    return a + b

add(1, 2)

6

In [6]:
add

<function __main__.add(a, b)>

## Decorators 101
A decorator is a callable that takes another function as argument (the decorated function).

In [7]:
def decorate(target_func):
    def inner():
        print("running inner()")
    return inner

@decorate
def target():
    print("running target()")

target()

running inner()


To summarize: the first crucial fact about decorators is that they have the power to replace the decorated function with a different one. The second crucial fact is that they are executed immediately when a module is loaded.

## When Python Executes Decorators

A key feature of decorators is that they run right after the decorated function is defined.

In [8]:
import sys
if "test_decorators" in sys.modules:
    del sys.modules["test_decorators"]

In [9]:
import test_decorators

running the function 'double'!


In [10]:
import sys
"test_decorators" in sys.modules

True

In [11]:
import inspect
lines = inspect.getsource(test_decorators)
print(lines)

import functools

def double(func):
    print("running the function 'double'!")
    @functools.wraps(func)
    def inner(*args, **kwargs):
        return 2 * func(*args, **kwargs)
    return inner

@double
def add(a, b):
    return a + b



function decorators are executed as soon as the module is imported, but the decorated functions only run when they are
explicitly invoked. This highlights the difference between what Pythonistas call import time and runtime.

## Decorator-Enhanced Strategy Pattern

In [12]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    ''' 5% discount for customers with 1000 or more fidelity points '''
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    ''' 10% discount for each LineItem with 20 or more units '''
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    '''7% discount for orders with 10 or more distinct items '''
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    ''' Select best discount available '''
    return max(promo(order) for promo in promos)



This example registers the function without changing it. Most decorators do change the decorated function. They usually do it by defining an inner function and returning it to replace the decorated function.  Code that uses inner functions almost always depends on closures to operate correctly. To understand closures, we need to take a step back a have a close look at how variable scopes work in Python.

## Variable Scope Rules

In [13]:
if "b" in globals():
    del b

def func(a):
    print(a)
    print(b)

try:
    func(1)
except Exception as e: # b is not defined
    print(e)

1
name 'b' is not defined


In [14]:
b = 2 # if we define b
func(1)

1
2


Now a more involved example.

In [15]:
import traceback

b = 2
def func(a):
    print(a)
    print(b)
    b = 3

try:
    func(1)
except:
    traceback.print_exc()

1


Traceback (most recent call last):
  File "<ipython-input-15-eb7c97220ba0>", line 10, in <module>
    func(1)
  File "<ipython-input-15-eb7c97220ba0>", line 6, in func
    print(b)
UnboundLocalError: local variable 'b' referenced before assignment


But the fact is, when Python compiles the body of the function, it decides that b is a local variable because it is assigned within the function. The generated bytecode reflects this decision and will try to fetch b from the local environment. Later, when the call func(1) is made, the body of f2 fetches and prints the value of the local variable a, but when
trying to fetch the value of local variable b it discovers that b is unbound.

In [16]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'globals()',
  '# we can implement something like this\ndef double(x):\n    return 2 * x\n\ndef add(a, b):\n    return a + b\n\ndouble(add(1, 2))',
  '# as this\ndef double(func):\n    def inner(*args, **kwargs):\n        return 2 * func(*args, **kwargs)\n    return inner\n\n@double\ndef add(a, b):\n    return a + b\n\nadd(1, 2)',
  'add',
  'import functools\n\ndef double(func):\n    @functools.wraps(func)\n    def inner(*args, **kwargs):\n        return 2 * func(*args, **kwargs)\n    return inner\n\n@double\ndef add(a, b):\n    return a + b\n\nadd(1, 2)',
  'add',
  'def decorate(target_func):\n    def inner():\n        print("running inner()")\n    return inner\n\n@decorate\ndef target():\n    print("running 

In [19]:
type(globals) # it's a

builtin_function_or_method

In [20]:
type(globals()) # that returns a

dict

In [22]:
"func" in globals()

True

In [23]:
globals()["func"]

<function __main__.func(a)>

In [24]:
type(locals)

builtin_function_or_method

In [25]:
type(locals())

dict

In [29]:
locals() == globals() # calling locals when we are in the global scope

True

In [32]:
def f():
    b = 2
    print(locals())

f()

{'b': 2}


Python compiles the body of the function, it decides that b is a local variable because it is assigned within the function. The generated bytecode reflects this decision and will try to fetch b from the local environment. 

This is not a bug, but a design choice: Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is local. This is much better than the behavior of JavaScript, which does not require variable declarations either, but if you do forget to declare that a variable is local (with var), you may clobber a global variable without knowing.


In [33]:
# if you want Python to treat b as a global variable in the function
def f():
    global b
    b = 2
    print(locals())

f()

{}


## Closures
In the blogosphere, closures are sometimes confused with anonymous functions. Actually, a closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not defined there. It does not matter
whether the function is anonymous or not; what matters is that it can access nonglobal variables that are defined outside of its body.

In [34]:
class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)


In [36]:
avg = Averager()

In [37]:
avg(10)

10.0

In [38]:
avg(11)

10.5

In [39]:
avg(12)

11.0

Now a functional implementation.

In [40]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager


In [41]:
avg = make_averager()

In [42]:
avg(10)

10.0

In [43]:
avg(11)

10.5

In [44]:
avg(12)

11.0

Note that series is a local variable of make_averager because the initialization series = [] happens in the body of that function. But when avg(10) is called, make_averager has already returned, and its local scope is long gone. Within averager, series is a free variable. This is a technical term meaning a variable that is not bound in the local scope. 

In [45]:
# # Python keeps the names of local and free variables in the __code__ attribute 
avg.__code__.co_varnames # get locals.

('new_value', 'total')

In [46]:
avg.__code__.co_freevars # and free variables.

('series',)

The binding for series is kept in the __closure__ attribute of the returned function avg. Each item in avg.__closure__ corresponds to a name in avg.__code__.co_free vars. These items are cells, and they have an attribute called cell_contents where
the actual value can be found. 

In [47]:
avg.__code__.co_freevars

('series',)

In [48]:
avg.__closure__

(<cell at 0x0000000004BB5BE8: list object at 0x0000000004C20248>,)

In [49]:
avg.__closure__[0].cell_contents

[10, 11, 12]

To summarize: a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

In [4]:
class A(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def add(self):
        return self.a + self.b

    def multiply(self):
        return self.a * self.b


In [5]:
a = A(2, 3)
a.add(), a.multiply()

(5, 6)

In [21]:
def func(a, b): # just having a play
    a = a
    b = b

    def add():
        return a + b

    def multiply():
        return a * b
    
    return locals()

a = func(2, 3)

In [22]:
a["a"], a["b"]

(2, 3)

In [23]:
a["add"](), a["multiply"]()

(5, 6)

## The nonlocal Declaration
Consider a broken higher-order function to calculate a running average without keeping all history.

In [24]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()

In [25]:
import traceback

try:
    avg(10)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-25-0656051f71a2>", line 4, in <module>
    avg(10)
  File "<ipython-input-24-dc311d6ae31a>", line 6, in averager
    count += 1
UnboundLocalError: local variable 'count' referenced before assignment


The problem is that the statement count += 1 actually means the same as count = count + 1, when count is a number or any immutable type. So we are actually assigning to count in the body of averager, and that makes it a local variable. The same problem affects the total variable. <br>
In the earlier example using series.append we took advantage of the fact that lists are mutable. But with immutable types like numbers, strings, tuples, etc., all you can do is read, but never update. To work around this, the nonlocal declaration was introduced in Python 3 (workarounds required for Python 2).

In [28]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()

In [29]:
avg(10)

10.0

In [30]:
avg(11)

10.5

In [31]:
avg(12)

11.0

## Implementing a Simple Decorator

In [36]:
import time

'''
Some shortcomings of this decorator:
* Doesn't support kwargs
* Should use functools.wraps decorator to copy the relevant attributes from func to clocked.
'''

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [33]:
@clock
def snooze(seconds):
    time.sleep(seconds)


In [34]:
snooze(1)

[0.99956610s] snooze(1) -> None


In [35]:
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

factorial(3)

[0.00000030s] factorial(1) -> 1
[0.00014390s] factorial(2) -> 2
[0.00016100s] factorial(3) -> 6


6

This is the typical behavior of a decorator: it replaces the decorated function with a new function that accepts the same arguments and (usually) returns whatever the decorated function was supposed to return, while also doing some extra processing.

## Decorators in the Standard Library
Python has three built-in functions that are designed to decorate methods: property, classmethod, and staticmethod.

In [1]:
# as an example
class A(object):
    counter = 0
    
    def __init__(self):
        A.increment_counter()
        self._is_ready = False

    @classmethod
    def get_counter(cls):
        return cls.counter

    @classmethod
    def increment_counter(cls):
        cls.counter += 1

    @property
    def is_ready(self):
        return self._is_ready

    @is_ready.setter
    def is_ready(self, is_ready):
        self.is_ready = is_ready

    @staticmethod
    def answer_to_life():
        return 42


With staticmethods, neither self (the object instance) nor  cls (the class) is implicitly passed as the first argument. They behave like plain functions except that you can call them from an instance or the class.

In [2]:
a = A()
b = A()

In [3]:
A.get_counter() # a class method

2

In [5]:
a.is_ready # an object attribute using @property

False

In [10]:
a.answer_to_life() # a staticmethod that we can call from either the class or object.

42

In [11]:
A.answer_to_life()

42

## Memoization with functools.lru_cache

In [13]:
import functools

def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)


@functools.lru_cache()
def fibonacci_with_cache(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [17]:
%timeit fibonacci(6)

2.3 µs ± 48.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [18]:
%timeit fibonacci_with_cache(6)

85.3 ns ± 0.717 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [19]:
%timeit fibonacci_with_cache(6)

84.9 ns ± 1.63 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Looks like there is something wrong with the lru_cache setup above...

## Generic Functions with Single Dispatch
Because we don’t have method or function overloading in Python, we can’t create variations of htmlize with different signatures for each data type we want to handle differently. A common solution in Python would be to turn htmlize into a dispatch function, with a chain of if/elif/elif calling specialized functions like htmlize_str, htmlize_int, etc. <br>

The new functools.singledispatch decorator in Python 3.4 allows each module to
contribute to the overall solution, and lets you easily provide a specialized function even
for classes that you can’t edit. 